In [ ]:
import os
import sys
sys.path.append(os.path.expanduser("~/Dropbox/Documents/Programming/Python/python-broadlink/"))

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
IR_DATA_DIR = os.path.expanduser("~/Dropbox/Documents/Programming/Python/py_rm_mini/ir_data/")
IR_DATA_EXT = '.irdump'

In [ ]:
import broadlink
# Doesn't seem to need this:
# broadlink.setup('ESSID', 'PASSWORD', 3)

In [ ]:
devices = broadlink.discover(timeout=5)

In [ ]:
devices, devices[0]

In [ ]:
devices[0].auth()

In [ ]:
dir(devices[0])

In [ ]:
if "Capture IR":
    b_start = widgets.Button(description="Capture IR")
    b_stop = widgets.Button(description="Save IR")
    device_name = widgets.Text(
        placeholder='Enter Device Name',
        description='Device Name:')
    button_name = widgets.Text(
        placeholder='Enter Button',
        description='Button:')

    for c in [b_start, b_stop, device_name, button_name]:
        display(c)

    def enter_learning(b):
        devices[0].enter_learning()

    def save_ir(b):
        path = os.path.join(IR_DATA_DIR, device_name.value, button_name.value + IR_DATA_EXT)
        devices[0].check_data()
        _irdat = devices[0].check_data()

        if not os.path.exists(os.path.dirname(path)):
            os.mkdir(os.path.dirname(path))

        with open(path, 'wb') as fh:
            fh.write(_irdat)

    b_start.on_click(enter_learning)
    b_stop.on_click(save_ir)

In [ ]:
if "Send IR":
    dev_strings = os.listdir(IR_DATA_DIR)
    def get_commands(device):
        return [x[:-len(IR_DATA_EXT)] 
                for x in os.listdir(os.path.join(IR_DATA_DIR, device)) 
                if IR_DATA_EXT in x]

    s_device = widgets.Select(
        options=dev_strings,
        value=dev_strings[0],
        description='Device:'
    )
    s_command = widgets.Select(
        options=get_commands(dev_strings[0]),
        description='Command:'
    )

    b_send = widgets.Button(description="Send IR")

    display(s_device, s_command, b_send)

    def send_command(b):
        path = os.path.join(IR_DATA_DIR, s_device.value, s_command.value + IR_DATA_EXT)
        with open(path, 'rb') as fh:
            devices[0].send_data(fh.read())

    def switch_device(b):
        s_command.options = get_commands(s_device.value)

    b_send.on_click(send_command)
    s_device.observe(switch_device)